In [46]:
from newspaper import Article
import pandas as pd
import re
import torch

In [2]:
def clean(x):
    return x.replace("(","").replace(")","").replace("{","").replace("}","").replace("[","").replace("]","").replace('"',"").replace("'","").replace("=","")
def _get_ngrams(n, text):
    """Calcualtes n-grams.

    Args:
      n: which n-grams to calculate
      text: An array of tokens

    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    return _get_ngrams(n, words)

In [3]:
url1="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=015&aid=0004435156&date=20201021&type=1&rankingSeq=1&rankingSectionId=104"
url2="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=296&aid=0000046738&date=20201011&type=1&rankingSeq=2&rankingSectionId=103"
url3="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=009&aid=0004678970&date=20201021&type=1&rankingSeq=5&rankingSectionId=103"
url4="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=025&aid=0003045159&date=20201021&type=1&rankingSeq=1&rankingSectionId=105"
url5="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=138&aid=0002093413&date=20201021&type=1&rankingSeq=2&rankingSectionId=105"
url6="https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=056&aid=0010920303&date=20201021&type=1&rankingSeq=3&rankingSectionId=105"

In [4]:
article=Article(url1, language='ko')
article.download()
article.parse()
text1=article.text
article=Article(url2, language='ko')
article.download()
article.parse()
text2=article.text
article=Article(url3, language='ko')
article.download()
article.parse()
text3=article.text
article=Article(url4, language='ko')
article.download()
article.parse()
text4=article.text
article=Article(url5, language='ko')
article.download()
article.parse()
text5=article.text
article=Article(url6, language='ko')
article.download()
article.parse()
text6=article.text

In [5]:
tgt_text1="그런데 왓츠앱의 경우 암호화 처리를 하기 때문에 왓츠앱을 통해 주고받은 대화 내용을 은행에서 확인하기가 쉽지 않다는 점이 문제가 돼 왔다. 그러나 은행 직원들은 왓츠앱을 통해 주변 사람들과 편하게 대화를 나눌 수 있기 때문에 회사 방침에도 불구하고 사용을 전면 중단하지는 않고 있다. 모간스탠리가 이번에 해고한 킹 대표 등 두 명 모두 왓츠앱을 통해 위법행위를 저지르지 않은 것으로 드러났다."
tgt_text2="건강수명을 누리려면 먼저 치매, 만성질환, 암 등 치료가 어렵고 투병기간이 긴 질병부터 예방해야 한다. 근육을 저축하고 키우기 위해서는 근력운동과 단백질 섭취, 비타민 D 섭취 등을 동시에 하는 것이 가장 좋다. 젊을 때부터 근력운동을 통해 근육의 힘을 많이 키워놓으면 크게 도움이 된다."
tgt_text3="롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 빅데이터 경영에 나선다. 데이터를 무기로 시장을 잠식하는 네이버와 카카오 등 IT공룡들의 공세에 맞서 강희태 유통BU장·롯데쇼핑 부회장 직속의 빅데이터 조직을 발족, 그간 각 계열사별로 따로 관리하던 소비데이터를 그룹 차원에서 한데 모아 분석하고 이에 맞는 맞춤형 쇼핑서비스를 선보인다는 목표다. 21일 롯데에 따르면 강 부회장이 이끄는 롯데 유통BU는 지난 1일 강 부회장 직속의 데이터 거버넌스 태스크포스를 출범하고, TF장이자 CDO로 윤영선 롯데정보통신 상무를 임명했다."
tgt_text4="8년간 의식장애로 인한 마비로 침대에 누워있거나 휠체어에 묶여 있던 네덜란드 30대 남성이 수면제 복용 20분 만에 정상능력을 회복해 다시 걷고 말하고, 먹을 수 있게 됐다. 한편 앞서도 혼수상태에 있던 환자가 수면제 복용 뒤 일시적으로 정상 능력을 회복했다는 연구결과가 있었다. 연구팀은 리처드 치료를 계기로 수면제를 활용해 뇌손상이나 혼수상태에 있는 환자를 영구적인 정상 상태로 회복시키는 방안을 연구하고 있다."
tgt_text5="예상보다 글로벌 가입자가 적게 늘었다. 대신 한국과 일본이 체면치레를 해줬다. 넷플릭스가 이번 분기 국내에서 상당한 수익을 벌고 갈 것으로 예상되면서 국회를 중심으로 한국시장 무임승차 논란이 다시 한번 불거질 전망이다."
tgt_text6="미국의 소행성 탐사선 '오시리스-렉스'가 20일 소행성 '101955 베누' 표면에 접지해 암석 표본을 채집하는 데 성공했습니다. 이번에 채취된 베누의 샘플은 2023년쯤 받아볼 수 있을 전망입니다. 나사는 이를 통해 베누의 구성 성분과 기원, 형성과정 등을 살피고 수분의 존재 여부 등도 조사하게 됩니다."

In [6]:
df=pd.DataFrame({"body":[text1,text2,text3,text4,text5,text6],"tgt":[tgt_text1,tgt_text2,tgt_text3,tgt_text4,tgt_text5,tgt_text6]})
df

,body,tgt
0,사진=연합뉴스 사진=연합뉴스\n\n미국 월스트리트에서 거액 연봉을 받던 트레이더들이...,그런데 왓츠앱의 경우 암호화 처리를 하기 때문에 왓츠앱을 통해 주고받은 대화 내용을...
1,"[사진=게티이미지뱅크] [사진=게티이미지뱅크]\n\n요즘은 ""오래사세요""(장수)보다...","건강수명을 누리려면 먼저 치매, 만성질환, 암 등 치료가 어렵고 투병기간이 긴 질병..."
2,롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 ...,롯데그룹이 갈수록 치열해지는 온·오프라인 유통시장 경쟁에서 승리하기 위해 본격적인 ...
3,졸피뎀. 연합뉴스 졸피뎀. 연합뉴스\n\n8년간 의식장애로 인한 마비로 침대에 누워...,8년간 의식장애로 인한 마비로 침대에 누워있거나 휠체어에 묶여 있던 네덜란드 30대...
4,[디지털데일리 권하영기자] 넷플릭스가 기대이하 성적을 냈다. 예상보다 글로벌 가입자...,예상보다 글로벌 가입자가 적게 늘었다. 대신 한국과 일본이 체면치레를 해줬다. 넷플...
5,사진출처: NASA 사진출처: NASA\n\n사진출처: NASA 사진출처: NASA...,미국의 소행성 탐사선 '오시리스-렉스'가 20일 소행성 '101955 베누' 표면에...


In [14]:
df[:2].to_csv("train_data.csv",index=False)
df[2:4].to_csv("valid_data.csv",index=False)
df[4:6].to_csv("test_data.csv",index=False)

In [3]:
import json

In [4]:
from konlpy.tag import Kkma
def load_data(row):
    kkma=Kkma()
    article=row["body"]
    abstract=row["tgt"]
#     article=article.text
#     abstract=tgt_text
    src_txt=kkma.sentences(article)
    tgt_txt=kkma.sentences(abstract)

    source=[{"tokens":kkma.morphs(clean(sen)),"sentence":clean(sen)} for sen in src_txt]
    tgt=[{"tokens":kkma.morphs(clean(sen)),"sentence":clean(sen)} for sen in tgt_txt]
    return source,tgt
def _format_to_lines(f):
    source, tgt = load_data(f)
    return {'src': source, 'tgt': tgt}

def format_to_lines():
    for corpus_type in ['valid', 'test', 'train']:
        df=pd.read_csv(corpus_type+"_data.csv")
        df.apply(_format_to_lines,axis=1).to_json(corpus_type+"_data.json",orient="records")
format_to_lines()

In [32]:
with open("test_data.json","rb") as f:
    file=json.load(f)
file

[{'src': [{'tokens': ['디지털',
     '데일리',
     '권',
     '하영',
     '기자',
     '넷플릭스',
     '가',
     '기대',
     '이하',
     '성적',
     '을',
     '내',
     '었',
     '다',
     '.'],
    'sentence': ' 디지털 데일리 권 하영 기자 넷플릭스가 기대 이하 성적을 냈다.'},
   {'tokens': ['예상', '보다', '글로벌', '가입자', '가', '적', '게', '늘', '었', '다', '.'],
    'sentence': '예상보다 글로벌 가입자가 적게 늘었다.'},
   {'tokens': ['대신',
     '한국',
     '과',
     '일본',
     '이',
     '체면',
     '치레',
     '를',
     '하',
     '어',
     '주',
     '었',
     '다',
     '.'],
    'sentence': '대신 한국과 일본이 체면 치레를 해 줬다.'},
   {'tokens': ['넷플릭스',
     '가',
     '특별히',
     '언급',
     '하',
     '며',
     '기뻐하',
     'ㄹ',
     '정도',
     '이',
     '다',
     '.'],
    'sentence': '넷플릭스가 특별히 언급하며 기뻐할 정도다.'},
   {'tokens': ['국내',
     '넷',
     '플릭스',
     '유료',
     '가입자',
     '는',
     '330',
     '만',
     '명',
     '으로',
     '고공',
     '행진',
     '중이',
     '이',
     '다',
     '.'],
    'sentence': '국내 넷 플릭스 유료 가입자는 330만명으로 고공 행진 중이다.'},
   {'tokens': ['일각',


In [7]:
import itertools
import json
import os

import subprocess
import time
from os.path import join as pjoin

In [8]:

def cal_rouge(evaluated_ngrams, reference_ngrams):
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))
    return {"f": f1_score, "p": precision, "r": recall}


def combination_selection(doc_sent_list, abstract_sent_list, summary_size):
    def _rouge_clean(s):
        return re.sub(r'[^a-zA-Z0-9가-힣 ]', '', s)

    max_rouge = 0.0
    max_idx = (0, 0)
    abstract = sum(abstract_sent_list, [])
    abstract = _rouge_clean(' '.join(abstract)).split()
    sents = [_rouge_clean(' '.join(s)).split() for s in doc_sent_list]
    evaluated_1grams = [_get_word_ngrams(1, [sent]) for sent in sents]
    reference_1grams = _get_word_ngrams(1, [abstract])
    evaluated_2grams = [_get_word_ngrams(2, [sent]) for sent in sents]
    reference_2grams = _get_word_ngrams(2, [abstract])

    impossible_sents = []
    for s in range(summary_size + 1):
        combinations = itertools.combinations([i for i in range(len(sents)) if i not in impossible_sents], s + 1)
        for c in combinations:
            candidates_1 = [evaluated_1grams[idx] for idx in c]
            candidates_1 = set.union(*map(set, candidates_1))
            candidates_2 = [evaluated_2grams[idx] for idx in c]
            candidates_2 = set.union(*map(set, candidates_2))
            rouge_1 = cal_rouge(candidates_1, reference_1grams)['f']
            rouge_2 = cal_rouge(candidates_2, reference_2grams)['f']

            rouge_score = rouge_1 + rouge_2
            if (s == 0 and rouge_score == 0):
                impossible_sents.append(c[0])
            if rouge_score > max_rouge:
                max_idx = c
                max_rouge = rouge_score
    return sorted(list(max_idx))


def greedy_selection(doc_sent_list, abstract_sent_list, summary_size):
    def _rouge_clean(s):
        return re.sub(r'[^a-zA-Z0-9가-힣 ]', '', s)

    max_rouge = 0.0
    abstract = sum(abstract_sent_list, [])
    abstract = _rouge_clean(' '.join(abstract)).split()
    sents = [_rouge_clean(' '.join(s)).split() for s in doc_sent_list]
    evaluated_1grams = [_get_word_ngrams(1, [sent]) for sent in sents]
    reference_1grams = _get_word_ngrams(1, [abstract])
    evaluated_2grams = [_get_word_ngrams(2, [sent]) for sent in sents]
    reference_2grams = _get_word_ngrams(2, [abstract])

    selected = []
    for s in range(summary_size):
        cur_max_rouge = max_rouge
        cur_id = -1
        for i in range(len(sents)):
            if (i in selected):
                continue
            c = selected + [i]
            candidates_1 = [evaluated_1grams[idx] for idx in c]
            candidates_1 = set.union(*map(set, candidates_1))
            candidates_2 = [evaluated_2grams[idx] for idx in c]
            candidates_2 = set.union(*map(set, candidates_2))
            rouge_1 = cal_rouge(candidates_1, reference_1grams)['f']
            rouge_2 = cal_rouge(candidates_2, reference_2grams)['f']
            rouge_score = rouge_1 + rouge_2
            if rouge_score > cur_max_rouge:
                cur_max_rouge = rouge_score
                cur_id = i
        if (cur_id == -1):
            return selected
        selected.append(cur_id)
        max_rouge = cur_max_rouge

    return sorted(selected)

In [6]:
from pytorch_pretrained_bert import BertTokenizer

In [44]:
class BertData():
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src, tgt, oracle_ids):

        if (len(src) == 0):
            return None
        original_src_txt = list(map(lambda x:x["sentence"],src))

        labels = [0] * len(list(map(lambda x:x["tokens"],src)))
        for l in oracle_ids:
            labels[l] = 1

        if (len(labels) == 0):
            return None

        src_txt = original_src_txt
        # text = [' '.join(ex['src_txt'][i].split()[:self.args.max_src_ntokens]) for i in idxs]
        # text = [_clean(t) for t in text]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = self.tokenizer.tokenize(text)
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = labels[:len(cls_ids)]

        tgt_txt = '<q>'.join(list(map(lambda x:x["sentence"],tgt)))
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt


def format_to_bert(oracle_mode): 
    for corpus_type in ['train', 'valid', 'test']:
        _format_to_bert((corpus_type+"_data.json",oracle_mode, corpus_type+"_data.bert.pt"))


def _format_to_bert(params):
    json_file,oracle_mode,save_file = params
#     if (os.path.exists(save_file)):
#         logger.info('Ignore %s' % save_file)
#         return

    bert = BertData()

#     logger.info('Processing %s' % json_file)
    jobs = json.load(open(json_file))
    datasets = []
    for d in jobs:
        source, tgt = d['src'], d['tgt']
        source_tokens=list(map(lambda x:x["tokens"],source))
        tgt_tokens=list(map(lambda x:x["tokens"],tgt))
        if (oracle_mode == 'greedy'):
            oracle_ids = greedy_selection(source_tokens, tgt_tokens, 3)
            
        elif (oracle_mode == 'combination'):
            oracle_ids = combination_selection(source_tokens, tgt_tokens, 3)
        b_data = bert.preprocess(source, tgt, oracle_ids)
        if (b_data is None):
            continue
        indexed_tokens, labels, segments_ids, cls_ids, src_txt, tgt_txt = b_data
        b_data_dict = {"src": indexed_tokens, "labels": labels, "segs": segments_ids, 'clss': cls_ids,
                       'src_txt': src_txt, "tgt_txt": tgt_txt}
        datasets.append(b_data_dict)
#     logger.info('Saving to %s' % save_file)
    torch.save(datasets, save_file)
#     datasets = []
#     gc.collect()







In [47]:
format_to_bert("greedy")

In [48]:
model=torch.load("test_data.bert.pt")
model

[{'src': [101,
   1166,
   32035,
   12799,
   87772,
   44492,
   21175,
   13926,
   1163,
   97090,
   11830,
   49828,
   13045,
   59365,
   90423,
   18463,
   28000,
   69433,
   44429,
   11376,
   1163,
   32035,
   14266,
   12398,
   35132,
   52250,
   76599,
   58380,
   17986,
   119,
   102,
   101,
   1174,
   61701,
   15046,
   74437,
   13988,
   50071,
   66074,
   20966,
   62776,
   37906,
   1175,
   33645,
   12756,
   15051,
   1165,
   63277,
   20001,
   119,
   102,
   101,
   78845,
   47529,
   12211,
   23488,
   11112,
   1177,
   40389,
   15463,
   1177,
   32035,
   53890,
   11643,
   46957,
   1175,
   97090,
   17986,
   119,
   102,
   101,
   90423,
   18463,
   28000,
   69433,
   44429,
   11376,
   1179,
   49904,
   12756,
   59415,
   18501,
   1174,
   46059,
   37031,
   22884,
   1163,
   32035,
   97077,
   33645,
   15168,
   81463,
   12265,
   12261,
   119,
   102,
   101,
   54602,
   12756,
   31516,
   90423,
   18463,
   1180,
  